In [1]:
import pandas as pd

players_df = pd.read_csv("nba-players.csv")
season_df = pd.read_csv("nba_extra.csv")

In [2]:
players_df = players_df.drop(columns=["College Link"])
players_df = players_df.drop("Unnamed: 0", axis=1)

In [3]:
def clean_player_name(name):
    if "\\" in name:
        return name.split("\\")[0].strip()
    return name.strip()


season_df["Player"] = season_df["Player"].apply(clean_player_name)

In [4]:
players_df.head(2)

,Name,Start Year,End Year,Position,Height,Weight,Birthday,Colleges
0,Player,1991,1995,F-C,10-Jun,240.0,24-Jun-68,Duke
1,Alaa Abdelnaby,1969,1978,C-F,9-Jun,235.0,7-Apr-46,Iowa State


In [5]:
season_df.head(1)

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Alex Abrines,SG,24,OKC,75,8,1134,115,291,...,0.848,26,88,114,28,38,8,25,124,353


### Merge season_df and players_df with a left join

Merge using a left outer join, that means, we want to have all the stats information, but if there are missing values that can't be matched from players_df, we want to set those season values as null.

Store the results from the merge in the variable df.

In [6]:
df = season_df.merge(players_df, how="left", left_on="Player", right_on="Name")
df.sample(5)

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,PF,PTS,Name,Start Year,End Year,Position,Height,Weight,Birthday,Colleges
180,148,James Ennis,SF,27,DET,27,8,551,74,162,...,53,202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,287,Jake Layman,SF,23,POR,35,1,160,14,47,...,13,34,Jake Layman,1972.0,1978.0,G,1-Jun,180.0,24-Dec-48,"Phoenix College, USC"
234,196,Montrezl Harrell,C,24,LAC,76,3,1293,348,548,...,143,836,Montrezl Harrell,2012.0,2014.0,C,10-Jun,275.0,12-Feb-89,Kentucky
318,256,Dakari Johnson,C,22,OKC,31,6,161,22,39,...,27,55,Dakari Johnson,1996.0,1996.0,G,1-Jun,170.0,26-Oct-65,Michigan State
581,472,Mirza Teletovic,PF,32,MIL,10,0,159,25,57,...,14,71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### How many rows couldn't be matched?

In [7]:
df["Name"].isna().sum()

np.int64(72)

### Extract and make a list of the names of the players that couldn't be matched

In [8]:
names = list(df.loc[df["Name"].isna(), "Player"].values)
names[:5]

['Alex Abrines', 'Omer Asik', 'Omer Asik', 'Omer Asik', 'Davis Bertans']

### Remove unnecessary columns

In [9]:
columns_to_drop = ["FG%","3P%","eFG%","FT%", "PF", "TOV", "BLK", "STL", "AST", "TRB", "DRB", "ORB", "FTA", "FT", "2P%", "2PA", "2P", "3PA", "3P"]

df.drop(columns=columns_to_drop, inplace=True)

In [10]:
df.head(3)

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,PTS,Name,Start Year,End Year,Position,Height,Weight,Birthday,Colleges
0,1,Alex Abrines,SG,24,OKC,75,8,1134,115,291,353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Quincy Acy,PF,27,BRK,70,8,1359,130,365,411,Quincy Acy,1976.0,1988.0,C-F,9-Jun,210.0,19-Jul-54,Oklahoma
2,3,Steven Adams,C,24,OKC,76,76,2487,448,712,1056,Steven Adams,1987.0,1997.0,F-G,7-Jun,215.0,22-Jul-64,Syracuse


### Rename teams to their full names

In [11]:
df_copy = df.copy()

In [12]:
team_mapping = {
    "OKC": "Oklahoma City Thunder",
    "DAL": "Dallas Mavericks",
    "BRK": "Brooklyn Nets",
    "SAC": "Sacramento Kings",
    "NOP": "New Orleans Pelicans",
    "MIN": "Minnesota Timberwolves",
    "SAS": "San Antonio Spurs",
    "IND": "Indiana Pacers",
    "MEM": "Memphis Grizzlies",
    "POR": "Portland Trail Blazers",
    "CLE": "Cleveland Cavaliers",
    "LAC": "Los Angeles Clippers",
    "PHI": "Philadelphia 76ers",
    "HOU": "Houston Rockets",
    "MIL": "Milwaukee Bucks",
    "NYK": "New York Knicks",
    "DEN": "Denver Nuggets",
    "ORL": "Orlando Magic",
    "MIA": "Miami Heat",
    "PHO": "Phoenix Suns",
    "GSW": "Golden State Warriors",
    "CHO": "Charlotte Hornets",
    "DET": "Detroit Pistons",
    "ATL": "Atlanta Hawks",
    "WAS": "Washington Wizards",
    "LAL": "Los Angeles Lakers",
    "UTA": "Utah Jazz",
    "BOS": "Boston Celtics",
    "CHI": "Chicago Bulls",
    "TOR": "Toronto Raptors"
}

df['Team'] = df['Tm'].replace(team_mapping)
df.head(2)

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,PTS,Name,Start Year,End Year,Position,Height,Weight,Birthday,Colleges,Team
0,1,Alex Abrines,SG,24,OKC,75,8,1134,115,291,353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oklahoma City Thunder
1,2,Quincy Acy,PF,27,BRK,70,8,1359,130,365,411,Quincy Acy,1976.0,1988.0,C-F,9-Jun,210.0,19-Jul-54,Oklahoma,Brooklyn Nets


### Convert birthday to a datetime object

In [13]:
df['Birthday'] = pd.to_datetime(df['Birthday'])

C:\Users\User\AppData\Local\Temp\ipykernel_29208\3789564390.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Birthday'] = pd.to_datetime(df['Birthday'])


### Delete all players from the TOT team

In [14]:
df = df[df['Tm'] != 'TOT']

In [15]:
df[df['Tm'] == 'TOT']

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,PTS,Name,Start Year,End Year,Position,Height,Weight,Birthday,Colleges,Team


### What's the team with the most players in the league?

In [17]:
df['Team'].value_counts().idxmax()

'Houston Rockets'

### What's the team with the lowest FG?

In [22]:
df.groupby('Team')['FG'].sum().sort_values().head()

Team
Memphis Grizzlies    3013
Dallas Mavericks     3128
Atlanta Hawks        3130
Brooklyn Nets        3136
Utah Jazz            3139
Name: FG, dtype: int64

### What's the team with the best FG%?
FG% is defined as FG / FGA

In [23]:
fg_per_team = df.groupby('Team')[['FG', 'FGA']].sum()
fg_per_team.head()

,FG,FGA
Team,,
Atlanta Hawks,3130,7015
Boston Celtics,3141,6975
Brooklyn Nets,3136,7114
Charlotte Hornets,3197,7106
Chicago Bulls,3170,7285


In [26]:
fg_per_team['FG%'] = fg_per_team['FG'] / fg_per_team['FGA'] * 100
fg_per_team.sort_values('FG%', ascending=False).head(1)	

,FG,FGA,FG%
Team,,,
Golden State Warriors,3509,6981,50.265005


In [27]:
fg_per_team['FG%'].idxmax()

'Golden State Warriors'

### Find the best scorers in each team

Create a new dataframe containing the best scorers per team (by PTS, total points scored). The resulting dataframe should contain the columns Player, Team, Pos and PTS, and should be stored in the variable best_scorers_per_team. It should be sorted by PTS in descending mode.

In [35]:
df['Best Score per Team'] = df.groupby('Team')['PTS'].transform('max')
df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,Name,Start Year,End Year,Position,Height,Weight,Birthday,Colleges,Team,Best Score per Team
0,1,Alex Abrines,SG,24,OKC,75,8,1134,115,291,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,Oklahoma City Thunder,2028
1,2,Quincy Acy,PF,27,BRK,70,8,1359,130,365,...,Quincy Acy,1976.0,1988.0,C-F,9-Jun,210.0,2054-07-19,Oklahoma,Brooklyn Nets,1007
2,3,Steven Adams,C,24,OKC,76,76,2487,448,712,...,Steven Adams,1987.0,1997.0,F-G,7-Jun,215.0,2064-07-22,Syracuse,Oklahoma City Thunder,2028
3,4,Bam Adebayo,C,20,MIA,69,19,1368,174,340,...,Bam Adebayo,2019.0,2019.0,F,7-Jun,200.0,1997-02-01,Louisville,Miami Heat,1296
4,5,Arron Afflalo,SG,32,ORL,53,3,682,65,162,...,Arron Afflalo,2023.0,2024.0,G,5-Jun,215.0,2000-04-20,Kansas,Orlando Magic,1022


In [36]:
best_scorers_per_team = df.loc[df['PTS'] == df["Best Score per Team"], ['Player', 'Team', 'Pos', 'PTS']].sort_values(by='PTS', ascending=False)

best_scorers_per_team.head()

,Player,Team,Pos,PTS
303,LeBron James,Cleveland Cavaliers,PF,2251
232,James Harden,Houston Rockets,SG,2191
148,Anthony Davis,New Orleans Pelicans,PF,2110
628,Russell Westbrook,Oklahoma City Thunder,PG,2028
15,Giannis Antetokounmpo,Milwaukee Bucks,PF,2014


### Which team has the 'youngest squad', by average player age?

In [40]:
df.groupby('Team')['Birthday'].mean().sort_values(ascending=False).head()

Team
Los Angeles Lakers    2035-04-22 05:42:51.428571392
Miami Heat            2033-09-05 19:30:00.000000000
Memphis Grizzlies     2032-02-22 20:52:10.434782720
Charlotte Hornets     2032-01-30 12:00:00.000000000
Cleveland Cavaliers   2031-10-12 16:25:15.789473792
Name: Birthday, dtype: datetime64[ns]